In [76]:
import subprocess
import pandas as pd

In [29]:
model_name = 'models/26jun_4modes'
model_data = 'models-data/data_26jun.dat'

In [47]:
p = subprocess.Popen(['C:/Program Files (x86)/biogeme-2.4/biogeme-2.4/biogeme.exe', model_name,model_data], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line),
retval = p.wait()

b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'biogeme 2.4 [Sun Oct 25 23:01:47 WEST 2015]\r\n'
b'Michel Bierlaire, EPFL\r\n'
b'-- Compiled by biogeme@hotmail.com on MINGW32_NT-6.1\r\n'
b'See http://biogeme.epfl.ch\r\n'
b'                    !! CFSQP is available !!\r\n'
b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'\t"In every non-trivial program there is at least one bug."\r\n'
b'\r\n'
b'\r\n'
b'[12:10:12]patFileNames.cc:52  models/26jun_4modes.par does not exist\r\n'
b'[12:10:12]patFileNames.cc:56  Trying default.par instead\r\n'
b'[12:10:12]patBiogeme.cc:134  Read default.par\r\n'
b' Opening file models-data/data_26jun.dat\r\n'
b' Data  file... line 500\tMemory: 74 Kb\r\n'
b' Data  file... line 1000\tMemory: 150 Kb\r\n'
b' Data  file... line 1500\tMemory: 225 Kb\r\n'
b' Total obs.:   1595\r\n'
b' Total memory: 240.023 Kb\r\n'
b' Run time for data processing: 00:00\r\n'
b' Init loglike=-1899.53\r\n'
b'     gmax Iter   radius        f(x)     Status 

In [74]:
with open(model_name+'_res.mod','wb') as simmod:
    with open(model_name+'.res','rb') as res:
        searchlines = res.readlines()
        foundit = False
        for i, line in enumerate(searchlines):
            if foundit==False:
                if b'[SampleEnum]' in line:
                    print('Found it.')
                    foundit = True
                simmod.write(line)
            else:
                if line.decode("utf-8").find('0\r\n')==0:
                    print('Changed to 100.')
                    simmod.write(b'100\r\n')
                    foundit==False
                else:
                    simmod.write(line)

Found it.
Changed to 100.


In [75]:
p = subprocess.Popen(['C:/Program Files (x86)/biogeme-2.4/biogeme-2.4/biosim.exe', model_name+'_res',model_data], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line),
retval = p.wait()

b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'biogeme 2.4 [Sun Oct 25 23:01:47 WEST 2015]\r\n'
b'Michel Bierlaire, EPFL\r\n'
b'-- Compiled by biogeme@hotmail.com on MINGW32_NT-6.1\r\n'
b'See http://biogeme.epfl.ch\r\n'
b'                    !! CFSQP is available !!\r\n'
b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'\t"In every non-trivial program there is at least one bug."\r\n'
b'\r\n'
b'\r\n'
b'[11:59:00]patFileNames.cc:52  models/26jun_4modes_res.par does not exist\r\n'
b'[11:59:00]patFileNames.cc:56  Trying default.par instead\r\n'
b'[11:59:00]patBiogeme.cc:134  Read default.par\r\n'
b'Warning:  Lower bound on mu set to 1e-005\r\n'
b'Warning:   Value defined by gevMinimumMu in default.par\r\n'
b' Opening file models-data/data_26jun.dat\r\n'
b' Data  file... line 500\tMemory: 74 Kb\r\n'
b' Data  file... line 1000\tMemory: 150 Kb\r\n'
b' Data  file... line 1500\tMemory: 225 Kb\r\n'
b' Total obs.:   1595\r\n'
b' Total memory: 240.023 Kb\r\n'
b' 31%\

In [77]:
simres = pd.read_csv(model_name+'_res.enu',sep='\t')
simres.head()

,Choice_Id,P_choice,V_A1_TAXI,V_A2_TRANSIT,V_A3_OTHER,V_A4_FREE,P_A1_TAXI,Residual_A1_TAXI,P_A2_TRANSIT,Residual_A2_TRANSIT,...,Simul91,Simul92,Simul93,Simul94,Simul95,Simul96,Simul97,Simul98,Simul99,Simul100
0,1,0.542337,0.875659,-0.065145,0.085024,-9999.99,0.542337,0.457663,0.211682,-0.211682,...,2,2,1,3,1,1,2,1,3,1
1,1,0.542337,0.875659,-0.065145,0.085024,-9999.99,0.542337,0.457663,0.211682,-0.211682,...,1,1,1,1,1,1,1,1,3,3
2,1,0.542337,0.875659,-0.065145,0.085024,-9999.99,0.542337,0.457663,0.211682,-0.211682,...,3,3,1,2,1,2,3,1,3,1
3,1,0.542337,0.875659,-0.065145,0.085024,-9999.99,0.542337,0.457663,0.211682,-0.211682,...,1,1,2,1,2,2,3,2,2,3
4,1,0.542337,0.875659,-0.065145,0.085024,-9999.99,0.542337,0.457663,0.211682,-0.211682,...,2,2,1,2,1,1,1,1,2,1


In [92]:
simul_start = list(simres.columns).index('Simul1')
def count_correct(row):
    return list(row[simul_start:]).count(row.Choice_Id)

In [99]:
simres['Accuracy'] = simres.apply(count_correct,axis=1)

In [100]:
simres.groupby('Choice_Id')['Accuracy'].mean()

Choice_Id
1    53.010435
2    21.722222
3    25.910394
4    99.996063
Name: Accuracy, dtype: float64